In [1]:
# Arturo Argueta
# Luis Montenegro
# Gabriel Garcia

# Simulacion de los ordenadoores y servicos

In [2]:
import numpy as np

In [3]:
# Parámetros
lambda_llegadas = 40  # Tasa de llegadas (solicitudes/segundo)
mu_mountain = 100     # Tasa de servicio de Mountain Mega Computing (solicitudes/segundo)
mu_pizzita = 10       # Tasa de servicio de Pizzita por servidor (solicitudes/segundo)
tiempo_simulacion = 3600  # Tiempo total de simulación (1 hora = 3600 segundos)


In [4]:
# Función para generar los tiempos de llegada (Poisson) y tiempos de servicio (Exponencial)
def generar_tiempos_llegada(lambda_llegadas, tiempo_simulacion):
    tiempos_llegada = []
    tiempo_actual = 0
    while tiempo_actual < tiempo_simulacion:
        tiempo_llegada = np.random.exponential(1 / lambda_llegadas)
        tiempo_actual += tiempo_llegada
        if tiempo_actual < tiempo_simulacion:
            tiempos_llegada.append(tiempo_actual)
    return tiempos_llegada

def generar_tiempos_servicio(mu, num_solicitudes):
    return np.random.exponential(1 / mu, num_solicitudes)



In [5]:
def simular_mountain(tiempos_llegada, mu_mountain):
    tiempos_servicio = generar_tiempos_servicio(mu_mountain, len(tiempos_llegada))
    tiempo_servidor_ocupado = 0
    tiempo_cola = 0
    salida_solicitudes = []
    cola = []
    tiempo_servidor_ocupado_total = 0
    servidor_libre_en = 0

    for i, llegada in enumerate(tiempos_llegada):
        if llegada >= servidor_libre_en:  # Si el servidor está libre cuando llega la solicitud
            tiempo_servicio = tiempos_servicio[i]
            salida = llegada + tiempo_servicio
            salida_solicitudes.append(salida)
            servidor_libre_en = salida
            tiempo_servidor_ocupado_total += tiempo_servicio
        else:
            cola.append(llegada)
            tiempo_cola += servidor_libre_en - llegada
            tiempo_servicio = tiempos_servicio[i]
            salida = servidor_libre_en + tiempo_servicio
            salida_solicitudes.append(salida)
            servidor_libre_en = salida
            tiempo_servidor_ocupado_total += tiempo_servicio

    tiempo_desocupado = tiempo_simulacion - tiempo_servidor_ocupado_total
    return len(salida_solicitudes), tiempo_servidor_ocupado_total, tiempo_desocupado, tiempo_cola, salida_solicitudes

# Simulación de Pizzita Computing (Múltiples servidores)
def simular_pizzita(tiempos_llegada, mu_pizzita):
    tiempos_servicio = generar_tiempos_servicio(mu_pizzita, len(tiempos_llegada))
    servidores = []
    tiempo_servidor_ocupado_total = 0
    tiempo_cola_total = 0
    solicitudes_atendidas = 0
    salida_solicitudes = []
    
    for i, llegada in enumerate(tiempos_llegada):
        # Revisar si hay un servidor disponible o en uso
        servidores_disponibles = [servidor for servidor in servidores if servidor['libre_en'] <= llegada]
        if servidores_disponibles:
            servidor = servidores_disponibles[0]
        else:
            # Crear un nuevo servidor si no hay disponibles
            servidor = {'libre_en': llegada}
            servidores.append(servidor)

        # Calcular tiempo de servicio y asignarlo al servidor
        tiempo_servicio = tiempos_servicio[i]
        servidor['libre_en'] = max(servidor['libre_en'], llegada) + tiempo_servicio
        salida_solicitudes.append(servidor['libre_en'])
        solicitudes_atendidas += 1
        tiempo_servidor_ocupado_total += tiempo_servicio

        # Si hubo tiempo en cola
        if llegada < servidor['libre_en'] - tiempo_servicio:
            tiempo_cola_total += servidor['libre_en'] - tiempo_servicio - llegada

    tiempo_desocupado_total = tiempo_simulacion - tiempo_servidor_ocupado_total
    return solicitudes_atendidas, tiempo_servidor_ocupado_total, tiempo_desocupado_total, tiempo_cola_total, salida_solicitudes, len(servidores)

In [7]:
# Ejecutar la simulación para Mountain Mega Computing
tiempos_llegada = generar_tiempos_llegada(lambda_llegadas, tiempo_simulacion)
solicitudes_mountain, tiempo_ocupado_mountain, tiempo_desocupado_mountain, tiempo_cola_mountain, salidas_mountain = simular_mountain(tiempos_llegada, mu_mountain)

# Ejecutar la simulación para Pizzita Computing
solicitudes_pizzita, tiempo_ocupado_pizzita, tiempo_desocupado_pizzita, tiempo_cola_pizzita, salidas_pizzita, num_servidores = simular_pizzita(tiempos_llegada, mu_pizzita)

# Calcular métricas adicionales
tiempo_promedio_cola_mountain = tiempo_cola_mountain / solicitudes_mountain
tiempo_promedio_cola_pizzita = tiempo_cola_pizzita / solicitudes_pizzita
solicitudes_promedio_cola_mountain = tiempo_cola_mountain / tiempo_simulacion
solicitudes_promedio_cola_pizzita = tiempo_cola_pizzita / tiempo_simulacion

AttributeError: 'numpy.ndarray' object has no attribute 'pop'

## Resultados de las simulaciones

In [ ]:


# Mostrar resultados de la simulación para Mountain Mega Computing
print("---- Mountain Mega Computing (Servidor Único) ----")
print(f"Solicitudes atendidas: {solicitudes_mountain}")
print(f"Tiempo ocupado del servidor: {tiempo_ocupado_mountain:.2f} segundos")
print(f"Tiempo desocupado del servidor: {tiempo_desocupado_mountain:.2f} segundos")
print(f"Tiempo total en cola: {tiempo_cola_mountain:.2f} segundos")
print(f"Tiempo promedio en cola por solicitud: {tiempo_promedio_cola_mountain:.2f} segundos")
print(f"Solicitudes promedio en cola: {solicitudes_promedio_cola_mountain:.2f}")
print(f"Última solicitud salida en: {max(salidas_mountain):.2f} segundos")

# Mostrar resultados de la simulación para Pizzita Computing
print("\n---- Pizzita Computing (Múltiples Servidores) ----")
print(f"Solicitudes atendidas: {solicitudes_pizzita}")
print(f"Tiempo ocupado total de los servidores: {tiempo_ocupado_pizzita:.2f} segundos")
print(f"Tiempo desocupado total de los servidores: {tiempo_desocupado_pizzita:.2f} segundos")
print(f"Tiempo total en cola: {tiempo_cola_pizzita:.2f} segundos")
print(f"Tiempo promedio en cola por solicitud: {tiempo_promedio_cola_pizzita:.2f} segundos")
print(f"Solicitudes promedio en cola: {solicitudes_promedio_cola_pizzita:.2f}")
print(f"Número de servidores utilizados: {num_servidores}")
print(f"Última solicitud salida en: {max(salidas_pizzita):.2f} segundos")